In [1]:
import bonesis
import bonesis.aeon
import biodivine_aeon
import os
import csv

In [2]:
aeon = "ABA_B.aeon"
data_loc = "ABA_B.tsv"

In [3]:
aeon_bn = biodivine_aeon.BooleanNetwork.from_file(aeon)

aeon_bn.explicit_parameter_count()

rg = aeon_bn.to_graph()


print(len(rg.variable_names()))

data = {}
for node in rg.variable_names():
    data[node + "_OFF"] = {node:0}
    data[node + "_ON"] = {node:1}


print(data)

85
{'ABA_OFF': {'ABA': 0}, 'ABA_ON': {'ABA': 1}, 'ABH1_OFF': {'ABH1': 0}, 'ABH1_ON': {'ABH1': 1}, 'ABI1_OFF': {'ABI1': 0}, 'ABI1_ON': {'ABI1': 1}, 'ABI2_OFF': {'ABI2': 0}, 'ABI2_ON': {'ABI2': 1}, 'ADPRc_OFF': {'ADPRc': 0}, 'ADPRc_ON': {'ADPRc': 1}, 'ARP_Complex_OFF': {'ARP_Complex': 0}, 'ARP_Complex_ON': {'ARP_Complex': 1}, 'Actin_Reorganization_OFF': {'Actin_Reorganization': 0}, 'Actin_Reorganization_ON': {'Actin_Reorganization': 1}, 'AnionEM_OFF': {'AnionEM': 0}, 'AnionEM_ON': {'AnionEM': 1}, 'AquaporinPIP2_1_OFF': {'AquaporinPIP2_1': 0}, 'AquaporinPIP2_1_ON': {'AquaporinPIP2_1': 1}, 'AtRAC1_OFF': {'AtRAC1': 0}, 'AtRAC1_ON': {'AtRAC1': 1}, 'CIS_OFF': {'CIS': 0}, 'CIS_ON': {'CIS': 1}, 'CPK23_OFF': {'CPK23': 0}, 'CPK23_ON': {'CPK23': 1}, 'CPK3_21_OFF': {'CPK3_21': 0}, 'CPK3_21_ON': {'CPK3_21': 1}, 'CPK6_OFF': {'CPK6': 0}, 'CPK6_ON': {'CPK6': 1}, 'Ca2osc_OFF': {'Ca2osc': 0}, 'Ca2osc_ON': {'Ca2osc': 1}, 'CaIM_OFF': {'CaIM': 0}, 'CaIM_ON': {'CaIM': 1}, 'Closure_OFF': {'Closure': 0}, 'Clos

In [2]:
skip_list = []

# # ignoring experiments that didn't work for ABA_GA1_B : 94
# skip_list = [
#     19,20,66,68,73,75,77,78,79,80,
#     88,91,92,104,116,119,128,137,139,143,
#     151,156,159,163,167,168,172,176,183,195,
#     197,201,204,210,217,221,223,224,227,228,
#     237,238,242,260,262,267,270,275,306,361,
#     367,377,423,425,427,428,438,443,456,465,
#     468,469,470,473,474,480,26,35,67,84,
#     86,117,120,126,174,207,220,231,233,236,
#     244,252,254,256,266,290,292,313,320,417,
#     418,437,449,175
#     ]

# # ignoring experiments that didn't work for ABA_base_B :260
# skip_list = [
#     7,8,9,18,19,25,42,53,63,66,
#     67,71,79,80,86,91,92,98,104,112,
#     116,117,118,119,120,122,125,126,128,130,
#     133,137,139,143,145,153,156,157,159,160,
#     162,163,165,172,174,175,176,179,183,195,
#     197,200,204,207,217,221,243,244,256,260,
#     262,294,306,307,313,316,318,326,328,331,
#     333,355,362,365,367,371,373,375,377,379,
#     381,412,414,415,417,418,423,425,427,428,
#     429,434,437,438,443,445,454,456,465,480,
#     486,35,226,236,249,255,259,267,275,312,
#     314,319,345,347,349,352,354,368,370,372,
#     374,409,411,439,461,468,494,498,500,502,
#     10,30,32,39,43,56,87,115,127,154,
#     177,190,219,220,230,241,242,252,254,264,
#     270,288,305,317,356,376,382,386,402,424,
#     431,453,1,13,16,21,26,28,41,45,
#     48,50,62,65,68,70,73,74,75,77,
#     78,81,93,95,97,99,103,105,109,132,
#     134,140,144,150,152,155,158,161,164,173,
#     180,186,188,196,198,199,201,203,206,208,
#     210,211,212,213,215,225,231,232,233,234,
#     240,245,247,251,253,280,282,284,287,293,
#     295,299,338,340,358,396,398,405,407,416,
#     420,422,441,449,462,466,469,471,472,473,
#     475,476,478,482,484,488,490,492,496,147
# ]

True


In [5]:
def comment_removal(line:str) -> bool:
    if line.startswith("#"):
        print(line)
    return not line.startswith("#") and not line.isspace()

def get_bo(aeon, data, data_loc, skip_list):

    dom = bonesis.aeon.AEONDomain.from_aeon_file(aeon)

    bo = bonesis.BoNesis(dom, data)


    ID, SCORE, SOURCE, PERT, NODE, VALUE = 0, 1, 2, 3, 4, 5

    file = open(data_loc, "r")
    lines = filter(comment_removal, file)
    data_csv = csv.reader(lines, delimiter="\t")

    # skip the first row
    next(data_csv)

    i = 0
    for row in data_csv:

        # ignore any data with "Some"
        if row[VALUE] in ["Some", "Some/ON", "OFF/Some"]:
            continue

        if int(row[ID]) in skip_list:
            continue
        
        i+=1

        if row[VALUE] == "ON":
            node_state = row[NODE] + "_ON"
        elif row[VALUE] == "OFF":
            node_state = row[NODE] + "_OFF"
        else:
            print("Unexpected value", row[VALUE])
            raise Exception("Unexpected experiment output")

        # wild type
        if row[SOURCE] == "" and row[PERT] == "":
            bo.fixed(bo.obs(node_state))
            bo.all_fixpoints(bo.obs(node_state))

        # perturbations
        else:
            fixes_dict = {}
            if row[SOURCE] != "":
                # add source node values to the fixes
                source_str = row[SOURCE].split(",")
                for sth in source_str:
                    node, value = sth.strip().split("=")
                    fixes_dict[node] = int(value)
        
            if row[PERT] != "":
                # add other perturbations to the fixes
                pert_str = row[PERT].split(",")
                for sth in pert_str:
                    node, value_str = sth.strip().split(" ")
                    if value_str == "KO":
                        value = 0
                    elif value_str == "CA":
                        value = 1
                    else:
                        raise Exception("Perturbation should be KO or CA")
                    fixes_dict[node] = int(value)

            with bo.mutant(fixes_dict):
                bo.fixed(bo.obs(node_state))
                bo.all_fixpoints(bo.obs(node_state))

    print("incorporated", i, "experiments")

    return bo

In [6]:
bo = get_bo(aeon, data, data_loc, skip_list)

incorporated 177 perturbations


In [ ]:
# bo.is_satisfiable()

In [ ]:
model_name = "ABA_B"

i = 0
for bn in bo.boolean_networks(limit=10000):
    i += 1

    if not os.path.exists(f"{model_name}"):
        os.makedirs(f"{model_name}")
    bn.save(f"{model_name}/bn-solution_{i}.bnet")


Grounding...